In [14]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
raw_file = "/home/ec2-user/SageMaker/Team-5/Fiona/TN_allinone0112.csv"
df = pd.read_csv(raw_file)

/tmp/ipykernel_11605/2699864545.py:2: DtypeWarning: Columns (3,4,5,6,10,15,17,21,22,24,25,27,28,30,31,33,35,36,38,40,41,44,45,48,49,52,53,56,57,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,78,81,82,83,85,86,94,107,108,109,110,111,112,113,114,115,117,119,122,126,128,129,130,132,133,137,138,142,143,147,148,152,153,157,158,162,163,167,168,169,170,171,172,173,175,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,253,256,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,303,304,305,307,310,313,317,319,322,324,325,328,330,332,334,335,336,338,343,348,349,351,352,353,354,355,356,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,38

In [16]:
height_columns = df.columns[df.columns.str.contains('height', case=False)].tolist()
print("Columns with 'height' in the title:")
print(height_columns)
print("")
weight_columns = df.columns[df.columns.str.contains('weight', case=False)].tolist()
print("Columns with 'weight' in the title:")
print(weight_columns)
print('')
gluc_columns = df.columns[df.columns.str.contains('gluc', case=False)].tolist()
print("Columns with 'gluc' in the title:")
print(gluc_columns)
print('')

for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        print(f"Unique values of '{column}':")
        unique_values = df[column].unique()
        print(unique_values)
        print('')
        
filtered_df = df.select_dtypes(include=['float64', 'int64'])
# print('List of numerical columns:')
# print(filtered_df.columns.tolist())
        
print('')

print('Has mmol as value')
for column in df.columns:
    if any('mmol' in str(value).lower() for value in df[column]):
        print(column)
        
print('')        
print('Has unit other than mdgl and under 50')        
for column in df.columns:
    if 'gluc' in column.lower():
        numeric_values = pd.to_numeric(df[column], errors='coerce')
        mask = (numeric_values.shift(1) < 50) & ~df[column].shift(1).astype(str).str.contains('mgdl', case=False)
        if mask.any():
            print(column)

Columns with 'height' in the title:
['Heightcm', 'Heightin', 'IsHeightMeasured', 'Height', 'HeightCm', 'HowIsHeightObtained', 'AnthopometricsHeightCM', 'HeightNotDone']

Columns with 'weight' in the title:
['Weightkg', 'Weightlbs', 'IsWeightMeasured', 'Weight', 'WeightKg', 'HowIsWeightObtained', 'AnthopometricsWeightKg', 'ATGweight', 'WeightNotDone']

Columns with 'gluc' in the title:
['AlphaGlucosidaseInhibitors', 'OtherGlucoseLoweringAgents', 'AreThere3GlucoseValuesAvailabl', 'TotalOfHomeBloodGlucoseOver3Da', 'AvgRecordedFastingGlucosesmmol', 'AvgRecordedFastingGlucosesmgdl', 'AvgAllRecordedGlucosesMmoll', 'AvgAllRecordedGlucosesMgDL', 'LowestGlucosemgdl', 'LowestGlucosemmoll', 'HighestGlucosemgdl', 'HighestGlucosemmol', 'SeizuresFaintingLowBloodGlucos', 'HospitalizationsForHighGlucose', 'OGTTTimeToConsumeGlucola', 'Glucose', 'PreDoseGlucose', 'PreDoseGlucoseNotDone', 'GlucoseValue', 'GlucoseValueNotDone']

Unique values of 'AvgRecordedFastingGlucosesmmol':
[nan 'mg/dl' 'mmol/L']

Un


Has mmol as value
AvgRecordedFastingGlucosesmmol
AvgAllRecordedGlucosesMmoll
LowestGlucosemmoll
HighestGlucosemmol

Has unit other than mdgl and under 50
TotalOfHomeBloodGlucoseOver3Da
AvgRecordedFastingGlucosesmgdl
AvgAllRecordedGlucosesMgDL
LowestGlucosemgdl
HighestGlucosemgdl
OGTTTimeToConsumeGlucola


In [17]:
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

for column in df.columns:
    mask = (df[column].shift(-1) == 'mmol/L')
    numeric_values = pd.to_numeric(df.loc[mask, column], errors='coerce')
    df.loc[mask, column] = numeric_values.where(numeric_values < 40, numeric_values * 18.018)
    df.loc[mask, column].shift(-1, fill_value='')[:] = 'mg/dl'
    
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmol', '_unit').replace('MgDL', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)

columns_to_multiply = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses_unit']
for column in columns_to_multiply:
    mask = (df[column] < 40)
    df.loc[mask, column] *= 18.018
    df.loc[mask, column].shift(-1, fill_value='')[:] = 'mg/dl'

In [18]:
# column_description = df['Height_2'].describe()
# print(column_description)
specified_column = 'HowIsHeightObtained'
column_index = df.columns.get_loc(specified_column)

selected_columns = df.iloc[:, max(0, column_index - 1):column_index + 2]

print(selected_columns)

for column in selected_columns.columns:
    unique_values = selected_columns[column].unique()
    print(f"Unique values in {column}: {unique_values}")

        HeightCm HowIsHeightObtained SubjectUsingCGMS
0            NaN                 NaN              NaN
1            NaN                 NaN              NaN
2            NaN                 NaN              NaN
3            NaN                 NaN              NaN
4            NaN                 NaN              NaN
...          ...                 ...              ...
259163     164.0            Measured              NaN
259164     164.7            Measured              NaN
259165     165.8            Measured               No
259166     165.9            Measured               No
259167     162.5            Measured              NaN

[259168 rows x 3 columns]
Unique values in HeightCm: [    nan 165.9   182.9   180.3   180.5   181.2   178.    177.    185.3
 185.9   185.8   186.1   188.    176.5   177.5   173.3   174.2   175.
 175.7   171.6   197.3   197.8   195.5   165.5   161.1   186.4   187.9
 149.5   163.3   166.    158.5   164.7   164.3   164.4   165.    163.7
 145.9   148.6 

In [19]:
numeric_columns = df.select_dtypes(include=np.number).columns

for column in numeric_columns:
    lower_bound = df[column].quantile(0.0025)
    upper_bound = df[column].quantile(0.9975)
    df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), np.nan, df[column])

print(numeric_columns)

Index(['MaskID', 'Spring2019ConsentDateDay', 'Spring2019ConsentDateYear',
       'DateOfBloodDrawDay', 'DateOfBloodDrawYear', 'TimeOfCollectionHH',
       'TimeOfCollectionMM', 'DateReportedDay', 'DateReportedYear',
       'TimeResultsProcessedByLabHH',
       ...
       'GLU', 'INST', 'DR3', 'DR4', 'HLA', 'HLAa', 'HLAb', 'Unnamed: 660',
       'Height_cm', 'Weight_kg'],
      dtype='object', length=303)
